## Cleaning Data (API Called) 6950 data points

In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#read in CSV
df = pd.read_csv('movieDB_API_movies.csv') 

# drop useless column index-like column
df = df.drop(columns='Unnamed: 0') 

## Create Metric Columns
# Create Total Profit Column WITH values containing budget/revenue
df['gross_profit'] = round(df[(df['revenue'] != 0) & (df['budget'] != 0)]['revenue'] -df[(df['revenue'] != 0) & (df['budget'] != 0)]['budget'],2)
# Create ROI column
df['ROI (%)'] = round((df['gross_profit'] / df['budget']) * 100,2)

# there are 2677 with null gross_profit, we want to
df_profits = df[df['gross_profit'].notna()]

# pd.DataFrame(df_profits).to_csv('movieDB_API_movies_w_gross_profits_&_ROI.csv') #Turn into csv file for easy access

In [ ]:
df_profits.head() # this is what our data looks like

In [ ]:
df_profits.info() # 6950 no missing data apparently, 4273/6950 have gross_profit (61% of data can use profit metric)

## What are the genres?

In [ ]:
#showing first ten genres
df_profits['genres'].value_counts()[0:10] #we should keep the combinations... They're important

### MY ANALYSIS
# What genres do we have?
## 1625 unique combinations of genres

# Top 3 Genres?
## Comedy 552/1625 or 34%
## Drama 419/1625 or 26%
## (Drama + Romance) 195/1625 or 12%

In [ ]:
genre_count_list = []
pd.DataFrame(df['genres']).iloc[0]
# so far each entry is made up of strings

df['genres'].value_counts() # maybe we can see for each genres 

In [ ]:
# each movie you have has a profit_margin (ROI)
# you have 1 movie with a list of genres ['Action, Adventure','Fantasy'a]
# 657
df_profits.head()

## Preprocess Data to Calculate Average Weighted Budget for Each Genre

In [ ]:
import ast #one example
from tqdm import tqdm # progress bar

#Fix data type of Genre Column
df['genres'][0]
res = ast.literal_eval(df['genres'][0]) 

df_profits = df_profits.reset_index(drop=True)

res_list = [] #loop through
for i in df_profits['genres']:
    res = ast.literal_eval(i)
    res_list.append(res)

df_profits['genres'] = res_list
df_profits['genres']

In [ ]:
# count of all genres
genre_and_count ={}
for genres in df_profits.genres:
    for genre in genres:
        genre_and_count[genre] = genre_and_count.get(genre, 1) + 1 

In [ ]:
print(sum(genres_count.values()))
genres_count

In [ ]:
# graph of the number of genres
fig, ax = plt.subplots(figsize=(15,6))
plt.xticks(rotation=45)
plt.bar(genres_count.keys(), genres_count.values());

## Calculate Average Weighted Budget for Each Genre

In [ ]:
# get number of genres per row and divide it by the budget
df_profits['qty_of_genres'] = df_profits['genres'].map(lambda x: len(x)) # get len of each genre and create column
df_profits['weighted_budget']  = round(df_profits['budget'] / df_profits['qty_of_genres'],2) #divide budget/genre qty for weights

In [ ]:
#sum the weighted budget for each genre to average it by the number of individual genres in next cell
genre_wbudget ={} 
for index in df_profits.index:
    for genre in df_profits['genres'][index]:
        budget = df_profits['weighted_budget'][index] 
        genre_wbudget[genre] = genre_wbudget.get(genre,0) + budget
genre_wbudget

In [ ]:
# get average of the weighted budget by genre count
genre_and_weight_budget = {}
for genre, budget in genre_wbudget.items():
    genre_and_weight_budget.update({genre:budget / genre_and_count[genre]})
genre_and_weight_budget 

In [ ]:
plt.figure(figsize=(12,8));
plt.bar(genre_and_weight_budget.keys(),genre_and_weight_budget.values());
plt.title('Genres and their weighted average budget');
plt.xlabel('Genres');
plt.ylabel('Average Budget');
plt.xticks(rotation=45)
plt.show()

## Calculate Average Weighted ROI for Each Genre

In [ ]:
def reject_outliers(data, m=2):
    return data[abs(data['ROI (%)'] - np.mean(data['ROI (%)'])) < m * np.std(data['ROI (%)'])]

df_profits = reject_outliers(df_profits, m=2)
df_profits[df_profits['title'] == 'Paranormal Activity']

In [ ]:
df_profits['weighted_ROI']  = round(df_profits['ROI (%)'] / df_profits['qty_of_genres'],2) #divide budget/genre qty for weights
# sum the weight
# sum the weight / qty

In [ ]:
# trying to link genre to the ROI
genre_roi_sum = {}
for index in df_profits.index:
    for genre in df_profits['genres'][index]:
        roi = df_profits['weighted_ROI'][index] 
        genre_roi_sum[genre] = genre_roi_sum.get(genre,0) + roi    
genre_roi_sum

In [ ]:
unique_genres = list(genre_roi_sum.keys())
unique_genres

In [ ]:
genre_weighted_roi = {}
for genre, roi in genre_roi_sum.items():
    genre_weighted_roi.update({genre: roi / genre_and_count[genre]})
genre_weighted_roi

In [ ]:
plt.figure(figsize=(12,8));
plt.bar(genre_weighted_roi.keys(),genre_weighted_roi.values());
plt.title('Genres and their weighted average ROI');
plt.xlabel('Genres');
plt.ylabel('Average ROI');
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12,8));
plt.bar(genre_weighted_roi.keys(),genre_weighted_roi.values());
plt.title('Genres and their weighted average ROI');
plt.xlabel('Genres');
plt.ylabel('Average ROI');
plt.xticks(rotation=45)
plt.show()

In [ ]:
#we should turn the years into most recent so that we can give examples of excellent ROI
df_profits[df_profits['genres'].map(lambda x: 'Horror' in x)]['release_date']

In [ ]:
df_profits[df_profits['genres'].map(lambda x: 'Thriller' in x)]

In [ ]:
import datetime

In [ ]:
df_profits['release_date'] = pd.to_datetime(df_profits['release_date'])

In [ ]:
df_profits['release_year'] = df_profits['release_date'].map(lambda x: x.year)

In [ ]:
sns.boxplot(df_profits['release_year'])

In [ ]:
df_profits.columns

In [ ]:
horror_df = df_profits[df_profits['genres'].map(lambda x: 'Horror' in x)].groupby('release_year').mean().reset_index()
horror_df[(horror_df['release_year'] >= 2010) & (horror_df['release_year'] <= 2020)].reset_index()[['release_year','weighted_budget','weighted_ROI']]

In [ ]:
def genre_year(df, genre):
    genre_df = df[df['genres'].map(lambda x: genre in x)].groupby('release_year').mean().reset_index()
    a = genre_df[(genre_df['release_year'] >= 2010) & (genre_df['release_year'] <= 2019)].reset_index()[['release_year','weighted_budget','weighted_ROI']]
    list_of_genres =[]
    for i in range(len(a)):
        list_of_genres.append(genre)
        
    a['genre'] = list_of_genres
    return a

In [ ]:
genre_year(df_profits,'Horror')

In [ ]:
profit_genre_by_year =  [genre_year(df_profits, genre) for genre in unique_genres]
profit_genres_by_year = pd.concat(profit_genre_by_year)

In [ ]:
a = profit_genres_by_year.reset_index(drop=True)